In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'eurosat-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F918039%2F1663377%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240531%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240531T143629Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1de10bc7130c91bc58aad23d29e0389f74b8ebe5053bdf26ca9acc6d92f8603712efde83e7585e464ba7b0343599dd6aca1ae72abada1a85b8a9a517d8c57adaab35cae035847b9e9910b3df2f510f6aa012f96eb9f494dbac831d62c79be697196a5d935e71d7a36cde204f6d08392c04d51de7eb70c2d54144adadb2149632365296e2aada5e3292866ffebdcad55608d4b55a248966e5ac175cbab4c31d72ec25fcd08276b3d894fa3d6efbe6b4ab96b9b775f57c0826291e6e6721b8b7b53af9e4ff4ef52838169dfa324a9002d485d1afe548be04a4fcc8b8317cf426349b2ac033a2d55670b9f168f49046eca98b059ead0cc9d46911745179636b818a,satellite-image-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1544742%2F2546969%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240531%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240531T143629Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6d32d677987bb676aa84111c5b51f52abb1ee85688950356c33a204046c1c8a60f6efd34936cafdb28b65c11d90e7ca55ee1a03ae9836ba7f67df193a8be517da2d049c039227bfdaaf48e41e3dee41e78dced5fe61237820040eb72719d68b44b20970fc2950242a358dabcc1722161d9e99573bf0431af1ae2b5fe4732db4b9f924d79d4ec2514f49be817e4a4569c3f3df2fbb0662cc333206c9f841f06364828ae7ec2d90c4b921c693e246b2cb2198237bbb5583ff6852e984967e5d46f691ec95996410e8e8de50dd369148b20960403450c980de92ea0a8aedc9fcd299fac4ed861eae5f0da374a4c62fb1c8281ce3ca59b6e33695e3b2c1f153a1a34'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
!pip install tensorflow==2.8.0

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
print("TensorFlow version:", tf.__version__)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam  # For optimizer
from tensorflow.keras.losses import CategoricalCrossentropy  # For loss function

# Optional imports depending on your data preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # For image data augmentation




TensorFlow version: 2.8.0


In [ ]:
tf.__version__

'2.8.0'

In [ ]:
gpu_available = tf.config.list_physical_devices('GPU')
if gpu_available:
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [ ]:
# import numpy as np
# import pandas as pd
# import io
# import os
# from matplotlib import image as mpimg
# import matplotlib.pyplot as plt
# import seaborn
# import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.preprocessing import image_dataset_from_directory
# from sklearn.metrics import confusion_matrix, classification_report

# import warnings
# warnings.filterwarnings('ignore')


In [ ]:
# data_dir = '/kaggle/input/satellite-image-classification/data'
# labels = os.listdir(data_dir)
# labels

In [ ]:

# train_dataset = image_dataset_from_directory('../input/satellite-image-classification/data',
#                                              shuffle=True,
#                                              batch_size=32,image_size=(224,224))
# class_name =  train_dataset.class_names
# plt.figure(figsize = (10,10))
# for image , label in train_dataset.take(1):
#     for i in range(9) :
#         plt.subplot(3,3,i+1)
#         plt.imshow(image[i].numpy().astype("uint8"))
#         plt.title(class_name[label[i]])
#         plt.axis("off")

In [ ]:
# img_width, img_height = 224, 224
# target_size = (img_width,img_height)
# batch_size = 32

In [ ]:
# #Create a data generator for image augmentation
# datagen = ImageDataGenerator( samplewise_center=True,  # set each sample mean to 0
#                              rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
#                              zoom_range = 0.1, # Randomly zoom imag
#                              width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#                              height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#                              horizontal_flip=True,  # randomly flip images
#                              vertical_flip=False,
#                              rescale=1./255,#scale images
#                              validation_split=0.2) #split data, 80% for training and 20% for testing


# #create training set from folders
# train_data=datagen.flow_from_directory(data_dir,
#                                        target_size=(target_size),
#                                        batch_size=batch_size,
#                                        class_mode='categorical',
#                                        shuffle=True,subset='training')

# #create test set
# test_data=datagen.flow_from_directory(data_dir,
#                                        target_size=(target_size),
#                                        batch_size=batch_size,
#                                        class_mode='categorical',
#                                        shuffle=False,subset='validation')

In [ ]:
# input_shape=(224,224,3)

In [ ]:
# def custom_net(input_shape):#custnet_1
#     model = Sequential([
#       Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
#       MaxPooling2D((2, 2)),
#       Conv2D(64, (3, 3), activation='relu'),
#       MaxPooling2D((2, 2)),
#       Conv2D(128, (3, 3), activation='relu'),
#       MaxPooling2D((2, 2)),

#       Flatten(),

#       Dense(64, activation='relu'),
#       Dense(32, activation='relu'),
#       Dense(16, activation='relu'),
#       Dense(4, activation='softmax')
#   ])

#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model


In [ ]:
# def custom_net(input_shape=(224, 224, 3)):#custnet_2
#     model = Sequential([
#       Conv2D(64, (7, 7), activation='relu', input_shape=input_shape),
#       MaxPooling2D((2, 2)),
#       Flatten(),
#       Dense(4, activation='softmax')  # 4 classes for output
#   ])

#   # Compile the model
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#     return model


In [ ]:
# model = custom_net(input_shape)

# # Compile the model


In [ ]:
# history = model.fit(train_data,
#                     epochs=20,
#                     validation_data=test_data,
#                     verbose=1)

In [ ]:
# model.summary()

In [ ]:
# model.evaluate(test_data)


In [ ]:
# model.save('custnet.h5')

RESNET


In [ ]:
# import numpy as np
# import pandas as pd
# import io
# import os
# from matplotlib import image as mpimg
# import matplotlib.pyplot as plt
# import seaborn
# import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.preprocessing import image_dataset_from_directory
# from sklearn.metrics import confusion_matrix, classification_report

# import warnings
# warnings.filterwarnings('ignore')


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.layers import Dense, Flatten
# from tensorflow.keras.models import Model, Sequential

# resnet_base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# for layer in resnet_base_model.layers:
#     layer.trainable = False

# resnet_base_model = Model(inputs=resnet_base_model.input, outputs=resnet_base_model.layers[-1].output)

In [ ]:
# resnet_model = Sequential()
# resnet_model.add(resnet_base_model)
# resnet_model.add(Flatten())
# resnet_model.add(Dense(4, activation='sigmoid'))

In [ ]:
# resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Define early stopping callback
# early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
# resnet_history = resnet_model.fit(train_data,
#                                   validation_data=test_data,
#                                   epochs=40)

In [ ]:
# resnet_model.evaluate(test_data)


In [ ]:
# resnet_model.summary()

In [ ]:

# resnet_model.save('resnet_model.h5')

In [ ]:
import os

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib import image
import seaborn as sns

from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3" # Block Numa warning

In [ ]:
input_folder = "/kaggle/input/"
dataset_folder = input_folder + "eurosat-dataset/" + "EuroSAT/"

In [ ]:
column_names =  ["id", "path", "class_id", "class_name"]

training_df = pd.read_csv(dataset_folder + "train.csv", names=column_names)
validation_df = pd.read_csv(dataset_folder + "validation.csv", names=column_names)
test_df = pd.read_csv(dataset_folder + "test.csv", names=column_names)

In [ ]:
training_df_size = len(training_df)
validation_df_size = len(validation_df)
test_df_size = len(test_df)

total_dataset_size = training_df_size + validation_df_size + test_df_size

print(f"Training set size: {training_df_size} ({round(training_df_size / total_dataset_size * 100)}%)")
print(f"Validation set size: {validation_df_size} ({round(validation_df_size / total_dataset_size * 100)}%)")
print(f"Test set size: {test_df_size} ({round(test_df_size / total_dataset_size * 100)}%)")

Training set size: 18901 (70%)
Validation set size: 5401 (20%)
Test set size: 2701 (10%)


In [ ]:
rescaling_factor = 1. / 255


In [ ]:
training_generator = ImageDataGenerator(rescale=rescaling_factor)
test_generator = ImageDataGenerator(rescale=rescaling_factor)

In [ ]:
image_width = 224
image_height = 224
batch_size = 128

In [ ]:
training_set = training_generator.flow_from_dataframe(
    dataframe=training_df,
    x_col="path",
    y_col="class_name",
    directory=dataset_folder,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle="true",
)

validation_set = test_generator.flow_from_dataframe(
    dataframe=validation_df,
    x_col="path",
    y_col="class_name",
    directory=dataset_folder,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle="true",
)

test_set = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col="path",
    y_col="class_name",
    directory=dataset_folder,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

/opt/conda/lib/python3.10/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 1 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Found 18900 validated image filenames belonging to 10 classes.


/opt/conda/lib/python3.10/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 1 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Found 5400 validated image filenames belonging to 10 classes.
Found 2700 validated image filenames belonging to 10 classes.


/opt/conda/lib/python3.10/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 1 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
class_name_list = np.array(list(training_set.class_indices.keys()))
labels, count = np.unique(training_set.classes, return_counts=True)

print(pd.Series(count, index=class_name_list))

AnnualCrop              2100
Forest                  2100
HerbaceousVegetation    2100
Highway                 1750
Industrial              1750
Pasture                 1400
PermanentCrop           1750
Residential             2100
River                   1750
SeaLake                 2100
dtype: int64


In [ ]:
#accuracy 90%
# import tensorflow as tf
# from tensorflow.keras import layers, models

# input_shape = (224, 224, 3)
# n_classes = 10

# img_input = tf.keras.Input(shape=input_shape)

# x1 = layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(img_input)
# x2 = layers.MaxPooling2D((2, 2))(x1)

# x3 = layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x2)
# x4 = layers.MaxPooling2D((2, 2))(x3)

# x5 = layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x4)
# x6 = layers.MaxPooling2D((2, 2))(x5)

# x7 = layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x6)
# x8 = layers.MaxPooling2D((2, 2))(x7)

# x9 = layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x8)
# x10 = layers.MaxPooling2D((2, 2))(x9)

# x11 = layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x10)
# x12 = layers.MaxPooling2D((2, 2))(x11)

# x13 = layers.Flatten()(x12)

# x14 = layers.Dense(64, activation='relu')(x13)
# output = layers.Dense(n_classes, activation='softmax')(x14)

# model = models.Model(inputs=img_input, outputs=output)

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# model.summary()


In [ ]:
# #accuracy 95%
# import tensorflow as tf
# from tensorflow.keras import layers, models

# # Define the input shape
# input_shape = (224, 224, 3)
# num_classes = 10
# inputs = tf.keras.layers.Input(shape=input_shape)
# x = tf.keras.layers.Conv2D(32, (3,3), activation='relu',padding='same')(inputs)
# x = tf.keras.layers.MaxPooling2D((2,2))(x)
# x = tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding='same')(x)
# x = tf.keras.layers.MaxPooling2D((2,2))(x)
# x = tf.keras.layers.Conv2D(128, (3,3), activation='relu',padding='same')(x)
# x = tf.keras.layers.MaxPooling2D((2,2))(x)
# x = tf.keras.layers.Dropout(0.3) (x)
# x = tf.keras.layers.Conv2D(128, (3,3), activation='relu',padding='same')(x)
# x = tf.keras.layers.MaxPooling2D((2,2))(x)
# x = tf.keras.layers.Dropout(0.3) (x)
# x = tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding='same')(x)
# x = tf.keras.layers.MaxPooling2D((2,2))(x)
# x = tf.keras.layers.Conv2D(32, (3,3), activation='relu',padding='same')(x)
# x = tf.keras.layers.MaxPooling2D((2,2))(x)
# x = tf.keras.layers.GlobalAveragePooling2D()(x)
# x = tf.keras.layers.Dense(64, activation='relu')(x)
# outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
# model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:

# # accuracy 87
# inputs = tf.keras.Input(shape=(224, 224, 3))

# # Convolutional layers
# x = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(inputs)
# x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

# x = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
# x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

# x = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
# x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

# x = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
# x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

# # Flatten layer
# x = tf.keras.layers.Flatten()(x)

# # Fully connected layers
# x = tf.keras.layers.Dense(128, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.2)(x)

# x = tf.keras.layers.Dense(512, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.2)(x)

# x = tf.keras.layers.Dense(1024, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.2)(x)

# outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

# # Create model
# model = tf.keras.Model(inputs=inputs, outputs=outputs)


# # Print model summary
# model.summary()

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
import tensorflow as tf


inputs = tf.keras.Input(shape=(224, 224, 3))

# Convolutional layers
x = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

x = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

x = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

x = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

# Flatten layer
x = tf.keras.layers.Flatten()(x)

# Fully connected layers
x = tf.keras.layers.Dense(16, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)



outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_8 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 112, 112, 32)     0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 56, 56, 128)       7385

In [ ]:
# import tensorflow as tf

# input_shape = (224, 224, 3)

# inputs = tf.keras.Input(shape=input_shape)

# x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
# x = tf.keras.layers.MaxPooling2D((2, 2))(x)

# x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
# x = tf.keras.layers.MaxPooling2D((2, 2))(x)

# x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
# x = tf.keras.layers.MaxPooling2D((2, 2))(x)

# x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu')(x)
# x = tf.keras.layers.MaxPooling2D((2, 2))(x)

# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dropout(0.5)(x)
# x = tf.keras.layers.Dense(128, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.5)(x)
# x = tf.keras.layers.Dense(64, activation='relu')(x)

# outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

# model = tf.keras.Model(inputs=inputs, outputs=outputs)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=40, verbose=1, restore_best_weights=True)


In [ ]:
history = model.fit_generator(
    generator=training_set,
    steps_per_epoch=len(training_set),
    epochs=40,
    validation_data=validation_set,
    validation_steps=len(validation_set),
    callbacks=[checkpoint,early_stopping]
)

/tmp/ipykernel_26/2096050626.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/40
148/148 [==============================] - ETA: 0s - loss: 2.0006 - accuracy: 0.2174
Epoch 1: val_accuracy improved from -inf to 0.33426, saving model to best_model.h5
148/148 [==============================] - 211s 1s/step - loss: 2.0006 - accuracy: 0.2174 - val_loss: 1.5940 - val_accuracy: 0.3343
Epoch 2/40
148/148 [==============================] - ETA: 0s - loss: 1.7303 - accuracy: 0.3058
Epoch 2: val_accuracy improved from 0.33426 to 0.39870, saving model to best_model.h5
148/148 [==============================] - 70s 471ms/step - loss: 1.7303 - accuracy: 0.3058 - val_loss: 1.4828 - val_accuracy: 0.3987
Epoch 3/40
148/148 [==============================] - ETA: 0s - loss: 1.6564 - accuracy: 0.3398
Epoch 3: val_accuracy improved from 0.39870 to 0.50426, saving model to best_model.h5
148/148 [==============================] - 67s 450ms/step - loss: 1.6564 - accuracy: 0.3398 - val_loss: 1.4321 - val_accuracy: 0.5043
Epoch 4/40
148/148 [==============================] - ETA

In [ ]:
test_loss, test_accuracy = model.evaluate_generator(test_set, steps=len(test_set))
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

/tmp/ipykernel_26/785264972.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_accuracy = model.evaluate_generator(test_set, steps=len(test_set))


Test Loss: 0.6370012760162354
Test Accuracy: 0.8481481671333313
